# Week 1 Homework — Time, Clocks, and Latency

This homework is fully guided. Follow the numbered steps. If you have never measured latency before, don't worry—each task includes starter code, definitions, and self-checks.

- Time on task: 90–120 minutes
- Tools: Python 3.9+ (3.10+ recommended), standard library, matplotlib (optional for plots)
- How to run: In VS Code or Jupyter, run cells from top to bottom. Re-run a cell with Shift+Enter.
- What to submit: This completed notebook with your code, plots (or printed stats), and short written answers.

Submission checklist:
- [ ] All code cells run without errors top-to-bottom
- [ ] You reported p50, p95, p99, and max for Task 1
- [ ] You included at least one plot (histogram or CDF) in Task 1 (textual stats acceptable if plotting unavailable)
- [ ] You computed clock skew over time and proposed a correction plan in Task 2
- [ ] You answered the fan-out questions in Task 3 and, if you choose, ran the optional simulation

Grading rubric (10 pts total):
- Task 1 methodology and results (4 pts): warm-up (1), sample size ≥ 5,000 (1), correct percentiles (1), interpretation of tails (1)
- Task 2 drift simulation and proposal (3 pts): correct skew computation (1), threshold reasoning (1), correction strategy clarity (1)
- Task 3 tail latency reasoning (3 pts): correct intuition (1), two mitigations with trade-offs (2)

---

## Task 0 — Setup (read then run)
We will optionally use `matplotlib` for plots. If it is not installed, the code will fall back to text-only output.

- If you cannot install packages, you can still complete the assignment using printed summary statistics.
- If you can install packages: in a terminal, run `pip install matplotlib` once.

Run the cell below to set up helpers and imports.


In [ ]:
import math, time, json, statistics, random, sys

# Try to import matplotlib; continue without it if unavailable
try:
    import matplotlib.pyplot as plt
    HAVE_MPL = True
except Exception as e:
    HAVE_MPL = False
    print("matplotlib not available; proceeding without plots.")

# Utility: percentile by index on a sorted list
def percentile(sorted_values, q):
    """Return the q percentile (0..100) from a pre-sorted list using nearest-rank index."""
    if not sorted_values:
        return float('nan')
    q = max(0, min(100, q))
    idx = int(round((q/100) * (len(sorted_values)-1)))
    return sorted_values[idx]


## Task 1 — Measure a local latency distribution (guided)
Goal: Measure the time to perform a simple local operation thousands of times and examine its distribution, especially the tail (p95/p99).

What is latency? The time to complete one operation. We will measure in milliseconds (ms). Tails (p95/p99) show the slower end of the distribution.

Step 1: Choose an operation (pick one):
- JSON encode+decode a small object (default)
- Concatenate small strings
- Sum a Python list of small integers (already in memory)

Run the next cell to define the operation and perform a short warm-up.


In [ ]:
# Choose operation by setting OPERATION = 'json' | 'str' | 'sum'
OPERATION = 'json'  # change if you want

obj = {"a": 1, "b": [i for i in range(50)], "c": "x"*128}
nums = list(range(100))

# Define the operation function
if OPERATION == 'json':
    def do_op():
        s = json.dumps(obj)
        json.loads(s)
elif OPERATION == 'str':
    def do_op():
        s = "".join(["a" for _ in range(200)])
else:  # 'sum'
    def do_op():
        sum(nums)

# Warm-up: run operation without timing to prime caches and JIT-like effects
for _ in range(500):
    do_op()

print("Warm-up complete.")


Step 2: Collect N measurements using `time.perf_counter()` (a high-precision monotonic timer). Recommendations:
- Use at least N = 5,000
- Avoid other heavy processes while measuring
- Remove outliers only if you can justify it (we will keep all samples here)

Run the cell to measure and compute summary statistics.


In [ ]:
N = 6000  # at least 5000
runs_s = []
for _ in range(N):
    t0 = time.perf_counter()
    do_op()
    runs_s.append(time.perf_counter() - t0)

runs_ms = sorted([x * 1e3 for x in runs_s])

summary = {
    'count': len(runs_ms),
    'p50_ms': percentile(runs_ms, 50),
    'p95_ms': percentile(runs_ms, 95),
    'p99_ms': percentile(runs_ms, 99),
    'max_ms': max(runs_ms),
    'mean_ms': statistics.fmean(runs_ms),
}
print(summary)

# Self-checks
assert len(runs_ms) >= 5000, "Need at least 5,000 samples"
assert summary['p50_ms'] <= summary['max_ms'] + 1e-9


Step 3 (optional but recommended): Visualize the distribution.
- Histogram: shows frequency of latencies
- CDF: shows the percentile curve; p95 is the value where 95% of samples are faster

Run this cell. If `matplotlib` is unavailable, it will print text instead.


In [ ]:
if HAVE_MPL:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].hist(runs_ms, bins=50, color="#93c5fd", edgecolor="#1e40af")
    axes[0].set_title("Latency Histogram (ms)")
    axes[0].set_xlabel("ms"); axes[0].set_ylabel("count")

    # Empirical CDF
    xs = runs_ms
    ys = [i/(len(xs)-1) for i in range(len(xs))]
    axes[1].plot(xs, ys, color="#1e40af")
    axes[1].axhline(0.95, color="#f59e0b", linestyle='--', label='p95')
    axes[1].axhline(0.99, color="#ef4444", linestyle='--', label='p99')
    axes[1].set_title("Latency CDF"); axes[1].set_xlabel("ms"); axes[1].set_ylabel("fraction ≤ x")
    axes[1].legend()
    plt.show()
else:
    print("Plotting unavailable. Here are a few quantiles:")
    for q in (10, 25, 50, 75, 90, 95, 99):
        print(f"p{q}: {percentile(runs_ms, q):.4f} ms")


### Write-up (Task 1)
In 3–6 sentences:
- Compare p50 to p95 and p99. Are the tails much higher? Why might that be on a single machine? (Think: cache misses, background processes, Python GC, CPU frequency scaling.)
- If you repeated the experiment at a different time or on battery power, what might change and why?

---

## Task 2 — Simulate clock drift and propose a correction strategy
Background: Real clocks are imperfect. Drift is how fast a clock runs vs. real time (parts-per-million, ppm). Two machines at +30 ppm and −40 ppm will diverge over time.

Step 1: Simulate per-second advancement for one hour and record the skew (A − B).


In [ ]:
def simulate_drift(seconds=3600, ppm_a=30, ppm_b=-40):
    rate_a = 1 + ppm_a/1_000_000
    rate_b = 1 + ppm_b/1_000_000
    a = b = 0.0
    skew = []  # seconds difference (A-B)
    for _ in range(seconds):
        a += rate_a
        b += rate_b
        skew.append(a - b)
    return skew

skew_s = simulate_drift()
print(f"Skew after 1 hour: {skew_s[-1]*1e3:.2f} ms")
print(f"Max |skew| over the hour: {max(abs(x) for x in skew_s)*1e3:.2f} ms")


Step 2 (optional plot): Visualize skew growth over time and find when it exceeds a threshold (e.g., 100 ms).


In [ ]:
THRESHOLD_MS = 100.0
if HAVE_MPL:
    plt.figure(figsize=(6,3))
    plt.plot([s*1e3 for s in skew_s], color="#1e40af")
    plt.axhline(THRESHOLD_MS, color="#ef4444", linestyle='--', label='threshold')
    plt.title("Clock Skew Over Time (ms)")
    plt.xlabel("seconds"); plt.ylabel("ms")
    plt.legend(); plt.show()

# Find first time skew exceeds threshold
first_exceed = next((i for i, s in enumerate(skew_s) if abs(s*1e3) > THRESHOLD_MS), None)
print("First exceed (s):", first_exceed)


Step 3: Propose a correction plan. Consider:
- Periodic synchronization interval based on how fast skew grows
- Slewing (gradually adjusting rate) vs. stepping (jumping time) and their side-effects
- Impact on timeouts, log ordering, and user-visible behavior

### Write-up (Task 2)
In 4–8 sentences: describe your plan and justify how it meets an SLA of <100 ms skew. Note trade-offs (e.g., frequent syncs = network overhead; slewing = temporary drift of timers).

---

## Task 3 — Why p99 dominates in fan-out requests
Scenario: A front-end makes 20 parallel calls. The page cannot render until all complete, so the slowest call often dictates user-perceived latency.

Part A (reasoning): Explain why the maximum of many samples tends to be closer to the tail of the single-call distribution. Use your Task 1 stats.

Part B (optional simulation): Sample from your empirical distribution, take the max of 20 calls per trial, and compare the resulting distribution to single-call latencies.


In [ ]:
# Optional simulation of fan-out tails using empirical samples from Task 1
TRIALS = 2000
FANOUT = 20
if 'runs_ms' in globals() and len(runs_ms) >= 1000:
    import random
    maxes = []
    for _ in range(TRIALS):
        picks = random.choices(runs_ms, k=FANOUT)
        maxes.append(max(picks))
    maxes.sort()
    fanout_summary = {
        'p50_max_ms': percentile(maxes, 50),
        'p95_max_ms': percentile(maxes, 95),
        'p99_max_ms': percentile(maxes, 99),
    }
    print("Fan-out (20 parallel) max latency stats:", fanout_summary)
    if HAVE_MPL:
        plt.figure(figsize=(6,3))
        plt.hist(maxes, bins=40, color="#93c5fd", edgecolor="#1e40af")
        plt.title("Distribution of Max Latency (20 parallel calls)")
        plt.xlabel("ms"); plt.ylabel("count")
        plt.show()
else:
    print("Run Task 1 first to populate runs_ms, then re-run this cell.")


### Write-up (Task 3)
In 4–8 sentences: Explain why fan-out pushes you into the tail. Propose at least two mitigations and note trade-offs, e.g.,
- Hedged requests (pros: reduces tail; cons: extra load)
- Timeouts with fallback/caching (pros: better perceived latency; cons: possible stale data)
- Prioritizing critical sub-requests or rendering progressively

---

## Final reflection (optional, +0.5 pt bonus)
1–3 sentences: One surprise from your measurements this week and one question you have for next week.
